In [1]:
pip install ipytest

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Importa la biblioteca pandas para el manejo de datos
import pandas as pd

# Importa la biblioteca numpy para operaciones numéricas
import numpy as np

# Importa el módulo datasets de sklearn para cargar conjuntos de datos
from sklearn import datasets

# Importa la clase LogisticRegression de sklearn para la regresión logística
from sklearn.linear_model import LogisticRegression

# Importa la función train_test_split de sklearn para dividir los datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

# Importa la clase StandardScaler de sklearn para la normalización de datos
from sklearn.preprocessing import StandardScaler

# Importa la biblioteca pytest para pruebas unitarias
import pytest

# Importa la biblioteca ipytest para ejecutar pruebas en Jupyter Notebook
import ipytest

# Configura ipytest para que se ejecute automáticamente
ipytest.autoconfig()

In [2]:
# Lee el archivo CSV y lo carga en un DataFrame de pandas
dataset = pd.read_csv(r"../../data/processed/TCGA_GBM_LGG_Mutations_clean_v2.csv") # cambiar la ruta cuando se pruebe

# Crea un DataFrame de pandas con las columnas del dataset
dataset_df = pd.DataFrame(dataset, columns=dataset.columns)

# Convierte la columna 'Grade' a tipo 'category'
dataset_df["Grade"] = dataset_df["Grade"].astype('category')

In [3]:
class SimplePipeline:
    def __init__(self):
        self.frame = None
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
        self.model = None
        self.load_dataset()
    
    def load_dataset(self):
        # Lee el archivo CSV y lo carga en un DataFrame de pandas
        dataset = pd.read_csv(r"../../data/processed/TCGA_GBM_LGG_Mutations_clean_v2.csv")
        
        # Obtiene los nombres de las características excluyendo la columna 'Grade'
        self.feature_names = dataset.columns.drop('Grade')
        self.frame = dataset
        
        # Divide el dataset en conjuntos de entrenamiento y prueba
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.frame[self.feature_names], self.frame['Grade'], test_size=0.65, random_state=42)
        
    def train(self, algorithm=LogisticRegression):
        # Entrena el modelo utilizando el algoritmo especificado (por defecto, regresión logística)
        self.model = algorithm(solver='lbfgs', multi_class='auto')
        self.model.fit(self.X_train, self.y_train)
        
    def predict(self, input_data):
        # Realiza predicciones utilizando el modelo entrenado
        return self.model.predict(input_data)
        
    def get_accuracy(self):
        # Calcula y devuelve la precisión del modelo en el conjunto de prueba
        return self.model.score(X=self.X_test, y=self.y_test)
    
    def run_pipeline(self):
        """Método de ejecución para correr la pipeline varias veces."""
        self.load_dataset()
        self.train()


class PipelineWithFeatureEngineering(SimplePipeline):
    def __init__(self):
        super().__init__()
        self.scaler = StandardScaler()
        self.scaler.fit(self.X_train)
    
    def apply_scaler(self):
        # Aplica la normalización a los conjuntos de entrenamiento y prueba
        self.X_train = self.scaler.transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)
        
    def predict(self, input_data):
        # Normaliza los datos de entrada antes de realizar predicciones
        scaled_input_data = self.scaler.transform(input_data)
        return self.model.predict(scaled_input_data)
                  
    def run_pipeline(self):
        self.load_dataset()
        self.apply_scaler()
        self.train()

In [4]:
# Crea una instancia de la clase PipelineWithFeatureEngineering
pipeline = PipelineWithFeatureEngineering()

# Ejecuta la pipeline, que incluye la carga del dataset, la normalización y el entrenamiento del modelo
pipeline.run_pipeline()

# Obtiene la precisión del modelo en el conjunto de prueba
accuracy_score = pipeline.get_accuracy()

# Imprime la precisión del modelo
print(f'La precisión del modelo es: {accuracy_score}')

La precisión del modelo es: 0.9157706093189965


## Testing

In [5]:
@pytest.fixture
def pipeline():
    # Crea una instancia de la clase PipelineWithFeatureEngineering
    pl = PipelineWithFeatureEngineering()
    
    # Carga el conjunto de datos en la instancia de la pipeline
    pl.load_dataset()
    
    # Devuelve la instancia de la pipeline
    return pl

#### Creating the test

In [6]:
%%ipytest

def test_scaler_preprocessing_brings_x_train_mean_near_zero(pipeline):
    # Calcula la media original de X_train
    original_mean = pipeline.X_train.stack().mean()
    
    # Aplica la normalización a X_train
    pipeline.apply_scaler()
    
    # Verifica que la media original es mayor que la media de X_train después de la normalización
    assert original_mean > pipeline.X_train.mean()
    
    # Verifica que la media de X_train después de la normalización está cerca de 0
    assert np.isclose(pipeline.X_train.mean(), 0.0, atol=1e-3)

    # Imprime la media original y la media transformada de X_train
    print(f'La media de X_train original es: {original_mean}')
    print(f'La media de X_train transformada es: {pipeline.X_train.mean()}')

def test_scaler_preprocessing_brings_x_train_std_near_one(pipeline):
    # Aplica la normalización a X_train
    pipeline.apply_scaler()
    
    # Verifica que la desviación estándar de X_train después de la normalización está cerca de 1
    assert np.isclose(pipeline.X_train.std(), 1.0, atol=1e-3)
    
    # Imprime la desviación estándar de X_train transformada
    print(f'La desviación estándar de X_train transformada es: {pipeline.X_train.std()}')

..                                                                                           [100%]
2 passed in 0.09s


#### Hacer que el test fallé

In [13]:
%%ipytest

def test_scaler_preprocessing_brings_x_train_mean_near_zero(pipeline):
    # Calcula la media original de X_train
    original_mean = pipeline.X_train.stack().mean()
    
    # Aplica la normalización a X_train
    pipeline.apply_scaler()

    # Cambia la aserción para que falle
    assert original_mean < pipeline.X_train.mean()

    # Cambia el valor en isclose para que falle
    assert not np.isclose(pipeline.X_train.mean(), 1.0, atol=1e-3)

F                                                                                            [100%]
============================================ FAILURES =============================================
_____________________ test_scaler_preprocessing_brings_x_train_mean_near_zero _____________________

pipeline = <__main__.PipelineWithFeatureEngineering object at 0x0000019D65E84850>

    def test_scaler_preprocessing_brings_x_train_mean_near_zero(pipeline):
        original_mean = pipeline.X_train.stack().mean()
    
        pipeline.apply_scaler()
    
        # Changing the assertion, so it will fail
>       assert original_mean < pipeline.X_train.mean()
E       assert 2.562661251791687 < -9.477297996481667e-18
E        +  where -9.477297996481667e-18 = <built-in method mean of numpy.ndarray object at 0x0000019D65E2AC10>()
E        +    where <built-in method mean of numpy.ndarray object at 0x0000019D65E2AC10> = array([[-1.22988009, -0.97262153, -0.25356194, ..., -0.16580533,\n        -

In [14]:
%%ipytest

def test_scaler_preprocessing_brings_x_train_std_near_one(pipeline):
    # Añade una gran variación en los datos
    pipeline.X_train *= 1000

    # Aplica la normalización a X_train
    pipeline.apply_scaler()

    # Prueba con la tolerancia original
    assert np.isclose(pipeline.X_train.std(), 1.0, atol=1e-3)

F                                                                                            [100%]
============================================ FAILURES =============================================
______________________ test_scaler_preprocessing_brings_x_train_std_near_one ______________________

pipeline = <__main__.PipelineWithFeatureEngineering object at 0x0000019D65F1F0A0>

    def test_scaler_preprocessing_brings_x_train_std_near_one(pipeline):
        # Adding huge variation in the data
        pipeline.X_train *= 1000
    
        pipeline.apply_scaler()
    
        # Testing with the original tolerance
>       assert np.isclose(pipeline.X_train.std(), 1.0, atol=1e-3)
E       assert False
E        +  where False = <function isclose at 0x0000019D369F6C20>(1270.4740150219047, 1.0, atol=0.001)
E        +    where <function isclose at 0x0000019D369F6C20> = np.isclose
E        +    and   1270.4740150219047 = <built-in method std of numpy.ndarray object at 0x0000019D65EBAF70>()
E 